In [46]:
import pandas as pd
import numpy as np

In [186]:
invent = pd.read_csv('./input/MS-b1-inventory.csv')
sup = pd.read_csv('./input/MS-b1-supply.csv')

In [187]:
sell = pd.read_csv('./input/MS-b1-sell.csv')

In [91]:
sell['date'] = pd.to_datetime(sell['date'], format='%Y-%m-%d')
invent['date'] = pd.to_datetime(invent['date'], format='%Y-%m-%d')
sup['date'] = pd.to_datetime(sup['date'], format='%Y-%m-%d')

In [92]:
invent.head()

,date,apple,pen
0,2006-01-31,12157,811
1,2006-02-28,29859,2280
2,2006-03-31,42135,3317
3,2006-04-30,55235,4094
4,2006-05-31,66963,5158


In [207]:
sell.tail()

,date,sku_num
7348349,2015-12-31,1
7348350,2015-12-31,1
7348351,2015-12-31,1
7348352,2015-12-31,1
7348353,2015-12-31,1


In [93]:
sell.head()

,date,sku_num
0,2006-01-01,MS-b1-ap-48914c5b-14d2-4b20-bdaf-b2ff5d9f4f0c
1,2006-01-01,MS-b1-ap-6baf7287-3e6a-4728-a3b1-8613de51eef8
2,2006-01-01,MS-b1-ap-83d7b005-c7d9-4deb-93a2-a8f7606d02b5
3,2006-01-01,MS-b1-ap-9099c5de-028d-4b07-97be-6f6532b8674a
4,2006-01-01,MS-b1-ap-ee0ce843-a979-4798-a542-75a41d1610c3


In [188]:
sell['sku_num'] = sell['sku_num'].apply(lambda x: 0 if x.find('ap') == 6 else 1)

In [189]:
sell['sku_num'].value_counts()

0    6826064
1     522290
Name: sku_num, dtype: int64

In [114]:
sell.head()

,date,sku_num
0,2006-01-01,0
1,2006-01-01,0
2,2006-01-01,0
3,2006-01-01,0
4,2006-01-01,0


In [119]:
sell['date'].drop_duplicates().size

3652

In [190]:
daily = pd.crosstab(sell['date'], sell['sku_num'], margins = True)
daily = daily.drop(['All'])

In [194]:
daily.head()

sku_num,0,1,All
date,,,
2006-01-01,1815,156,1971
2006-01-02,1862,143,2005
2006-01-03,1880,171,2051
2006-01-04,1797,153,1950
2006-01-05,1942,133,2075


In [209]:
daily_new = pd.DataFrame(columns=['date', 'apple', 'pen'])
daily_new['date'] = sell['date'].drop_duplicates()
daily_new['apple'] = daily[0].values
daily_new['pen'] = daily[1].values
daily_new.index = range(0, daily[0].size)

In [210]:
daily_new.head()

,date,apple,pen
0,2006-01-01,1815,156
1,2006-01-02,1862,143
2,2006-01-03,1880,171
3,2006-01-04,1797,153
4,2006-01-05,1942,133


In [211]:
stor_daily_b1 = pd.DataFrame(columns=['date', 'apple', 'pen'])

length_sell = len(daily_new) - 1
length_spl = len(sup) - 1

curr_apple = 0
curr_pen = 0

for i in range(length_sell + 1):
    
    curr_date = daily_new['date'][i]
    for j in range(length_spl):
        if sup['date'][j] == curr_date:
            curr_apple = curr_apple + sup['apple'][j]
            curr_pen = curr_pen + sup['pen'][j]
            
    curr_apple = curr_apple - daily_new['apple'][i]
    curr_pen  = curr_pen - daily_new['pen'][i]
   
    stor_daily_b1.at[i, 'apple'] = curr_apple
    stor_daily_b1.at[i, 'pen'] = curr_pen
    stor_daily_b1.at[i, 'date'] = curr_date
    
stor_daily_b1.head(n=15)

,date,apple,pen
0,2006-01-01,33271,2574
1,2006-01-02,31409,2431
2,2006-01-03,29529,2260
3,2006-01-04,27732,2107
4,2006-01-05,25790,1974
5,2006-01-06,23892,1825
6,2006-01-07,21997,1687
7,2006-01-08,20161,1567
8,2006-01-09,18293,1442
9,2006-01-10,16414,1286


In [212]:
stor_daily_b1.tail(n=15)

,date,apple,pen
3637,2015-12-17,1561360,122725
3638,2015-12-18,1559493,122587
3639,2015-12-19,1557604,122440
3640,2015-12-20,1555727,122287
3641,2015-12-21,1553871,122150
3642,2015-12-22,1552105,122023
3643,2015-12-23,1550244,121878
3644,2015-12-24,1548281,121743
3645,2015-12-25,1546458,121597
3646,2015-12-26,1544601,121449


не хватает 31 декабря

ппреобразуем индексы ко времени

In [213]:
daily_new = stor_daily_b1

In [214]:
daily_new.index = sell['date'].drop_duplicates()

In [215]:
daily_new = daily_new.drop(['date'], axis=1)

In [216]:
daily_new.head()

,apple,pen
date,,
2006-01-01,33271,2574
2006-01-02,31409,2431
2006-01-03,29529,2260
2006-01-04,27732,2107
2006-01-05,25790,1974


In [148]:
sup.index = sup.date
sup = sup.drop(['date'], axis=1)

In [168]:
sup.head()

,apple,pen
date,,
2006-01-01,35086,2730
2006-01-15,35002,2625
2006-02-01,34963,2759
2006-02-15,34869,2715
2006-03-01,35188,2599


In [172]:
daily_new['apple'] = -daily_new['apple']

In [175]:
df = pd.concat([sup, daily_new]).sort_index()
df.head()

,apple,pen
date,,
2006-01-01,35086,2730
2006-01-01,1815,156
2006-01-02,1862,143
2006-01-03,1880,171
2006-01-04,1797,153


In [179]:
df['apple'].resample('D').sum()

date
2006-01-01    36901
2006-01-02     1862
2006-01-03     1880
2006-01-04     1797
2006-01-05     1942
2006-01-06     1898
2006-01-07     1895
2006-01-08     1836
2006-01-09     1868
2006-01-10     1879
2006-01-11     1796
2006-01-12     1915
2006-01-13     1920
2006-01-14     1872
2006-01-15    36824
2006-01-16     1894
2006-01-17     1908
2006-01-18     1840
2006-01-19     1972
2006-01-20     1861
2006-01-21     1823
2006-01-22     1910
2006-01-23     1834
2006-01-24     1849
2006-01-25     1927
2006-01-26     1882
2006-01-27     1797
2006-01-28     1848
2006-01-29     1874
2006-01-30     1878
              ...  
2015-12-02     1915
2015-12-03     1825
2015-12-04     1815
2015-12-05     1899
2015-12-06     1875
2015-12-07     1908
2015-12-08     1829
2015-12-09     1849
2015-12-10     1868
2015-12-11     1943
2015-12-12     1959
2015-12-13     1931
2015-12-14     1844
2015-12-15    36606
2015-12-16     1937
2015-12-17     1837
2015-12-18     1867
2015-12-19     1889
2015-12-20     